In [5]:
import numpy as np
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [11]:
actions = ['a', 'b', 'c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','space','backspace','1','2','3']

data = np.concatenate([
    np.load("C:/Users/kelly/IdeaProjects/Project mid/dataset/seq_a_1689161582.npy"),
    np.load("C:/Users/kelly/IdeaProjects/Project mid/dataset/seq_b_1689161582.npy"),
    np.load("C:/Users/kelly/IdeaProjects/Project mid/dataset/seq_c_1689161582.npy"),
    np.load("C:/Users/kelly/IdeaProjects/Project mid/dataset/seq_d_1689161582.npy"),
    np.load("C:/Users/kelly/IdeaProjects/Project mid/dataset/seq_e_1689161582.npy"),
    np.load("C:/Users/kelly/IdeaProjects/Project mid/dataset/seq_f_1689161582.npy"),
    np.load("C:/Users/kelly/IdeaProjects/Project mid/dataset/seq_g_1689161582.npy"),
    np.load("C:/Users/kelly/IdeaProjects/Project mid/dataset/seq_h_1689161582.npy"),
    np.load("C:/Users/kelly/IdeaProjects/Project mid/dataset/seq_i_1689161582.npy"),
    np.load("C:/Users/kelly/IdeaProjects/Project mid/dataset/seq_j_1689161582.npy"),
    np.load("C:/Users/kelly/IdeaProjects/Project mid/dataset/seq_k_1689161582.npy"),
    np.load("C:/Users/kelly/IdeaProjects/Project mid/dataset/seq_l_1689161582.npy"),
    np.load("C:/Users/kelly/IdeaProjects/Project mid/dataset/seq_m_1689161582.npy"),
    np.load("C:/Users/kelly/IdeaProjects/Project mid/dataset/seq_n_1689161582.npy"),
    np.load("C:/Users/kelly/IdeaProjects/Project mid/dataset/seq_o_1689161582.npy"),
    np.load("C:/Users/kelly/IdeaProjects/Project mid/dataset/seq_p_1689161582.npy"),
    np.load("C:/Users/kelly/IdeaProjects/Project mid/dataset/seq_q_1689161582.npy"),
    np.load("C:/Users/kelly/IdeaProjects/Project mid/dataset/seq_r_1689161582.npy"),
    np.load("C:/Users/kelly/IdeaProjects/Project mid/dataset/seq_s_1689161582.npy"),
    np.load("C:/Users/kelly/IdeaProjects/Project mid/dataset/seq_t_1689161582.npy"),
    np.load("C:/Users/kelly/IdeaProjects/Project mid/dataset/seq_u_1689161582.npy"),
    np.load("C:/Users/kelly/IdeaProjects/Project mid/dataset/seq_v_1689161582.npy"),
    np.load("C:/Users/kelly/IdeaProjects/Project mid/dataset/seq_w_1689161582.npy"),
    np.load("C:/Users/kelly/IdeaProjects/Project mid/dataset/seq_x_1689161582.npy"),
    np.load("C:/Users/kelly/IdeaProjects/Project mid/dataset/seq_y_1689161582.npy"),
    np.load("C:/Users/kelly/IdeaProjects/Project mid/dataset/seq_z_1689161582.npy"),
    np.load('C:/Users/kelly/IdeaProjects/Project mid/dataset/seq_space_1689328397.npy'),
    np.load('C:/Users/kelly/IdeaProjects/Project mid/dataset/seq_backspace_1689328397.npy'),
    np.load('C:/Users/kelly/IdeaProjects/Project mid/dataset/seq_1_1689329907.npy'),
    np.load('C:/Users/kelly/IdeaProjects/Project mid/dataset/seq_2_1689330070.npy'),
    np.load('C:/Users/kelly/IdeaProjects/Project mid/dataset/seq_3_1689330070.npy')
], axis=0)

data.shape

(16177, 30, 100)

In [7]:
x_data = data[:, :, :-1]
labels = data[:, 0, -1]

print(x_data.shape)
print(labels.shape)

(16177, 30, 99)
(16177,)


In [18]:
from tensorflow.keras.utils import to_categorical

y_data = to_categorical(labels, num_classes=len(actions))
y_data.shape

IndexError: index 31 is out of bounds for axis 1 with size 30

In [9]:
from sklearn.model_selection import train_test_split

x_data = x_data.astype(np.float32)
y_data = y_data.astype(np.float32)

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=2021)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

NameError: name 'y_data' is not defined

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(64, activation='relu', input_shape=x_train.shape[1:3]),
    Dense(1024, activation='relu'),#원래 1024가 아니라 32였음 
    Dense(len(actions), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

history = model.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    epochs=200,
    callbacks=[
        ModelCheckpoint('models/model.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=50, verbose=1, mode='auto')
    ]
)

In [ ]:
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots(figsize=(16, 10))
acc_ax = loss_ax.twinx()

loss_ax.plot(history.history['loss'], 'y', label='train loss')
loss_ax.plot(history.history['val_loss'], 'r', label='val loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

acc_ax.plot(history.history['acc'], 'b', label='train acc')
acc_ax.plot(history.history['val_acc'], 'g', label='val acc')
acc_ax.set_ylabel('accuracy')
acc_ax.legend(loc='upper left')

plt.show()

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
from tensorflow.keras.models import load_model

model = load_model('models/model.h5')

y_pred = model.predict(x_val)

multilabel_confusion_matrix(np.argmax(y_val, axis=1), np.argmax(y_pred, axis=1))